In [33]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import scipy.signal
import random
from sklearn.cluster import KMeans

test_data_file = {}

def select_test_files():
        # because we have 17,000 total audio files and we need 10% to test with
        for i in range(0 , 10):
            rand_spkr = random.randint(1,3)
#             print("rand_spkr: " , rand_spkr)
            rootdir = 'C:\\Users\\James Hoffman\\Desktop\\Speech_Recognition\\test_files\\' + str(rand_spkr)
            x = 0
            random_file = random.randint(0,500)
#             print("random file: " , random_file)
            for filename in os.listdir(rootdir):
                if(x == random_file):
                   test_data_file[filename] = rand_spkr
                x += 1
        
        print(test_data_file)

def display_graph_for_data(data , sampling_rate):
    plt.figure(figsize=(12 , 4))
    librosa.display.waveplot(data, sr=sampling_rate)
    plt.show()

'''
     takes the location of an audio file and does the fourier transform,
     flattens that and returns it as a single vector
'''    
def transform(audio):
    data, sampling_rate = librosa.load(audio, sr=4000)
    spectral = 10 * np.log10(np.abs(librosa.core.stft(data, n_fft=512, hop_length=80, window=scipy.signal.hanning)))
    print(spectral.shape)
    w, h = spectral.shape
    spectral = specztral.flatten('F')
    print(spectral.shape)
    return spectral


'''
     takes a single directory (one of the numbered one from like 1-30)
     and returns an np.array which contains the flattened data that can
     be fed into the kmeans algorithm
'''
def parse_directory(audio_directory , dir_num): 
    
    speaker_vector = []
    audio_directory = audio_directory

    x = 0
    
    for filename in os.listdir(audio_directory):
        
#         print(filename)
        
        if filename in test_data_file.keys():
            print("filename found")
            print(filename)
            print(test_data_file[filename])
            print(dir_num)
            if(dir_num == test_data_file[filename]):
                print("found one")
        
        x += 1
        if(x > 10):
            break
            
        flat_audio = transform(audio_directory + "\\" + filename).tolist()
        speaker_vector = speaker_vector + flat_audio
        
    return speaker_vector

'''
     takes a single directory (the one with all the data)
     and returns an np.array which has the all the np arrays that contain the
     vector for each of the numbered speakers in that directory
'''
def iterate_test_folders(rootdir):
    audio_vectors = []
    for subdir, dirs, files in os.walk(rootdir):
        for dir in dirs:
            print("iterating through directory: " , dir)
            p_d_ = parse_directory(rootdir + dir , dir)
            audio_vectors.append(p_d_)
            
    return audio_vectors

    
# select_test_files()

In [34]:
def get_vectors():
    rootdir = 'C:\\Users\\James Hoffman\\Desktop\\Speech_Recognition\\test_files\\'
    list_of_flattened_vectors = iterate_test_folders(rootdir)
    
    return list_of_flattened_vectors
    

In [35]:
def main():
    sample = get_vectors()
    print(len(sample))
    print(any(isinstance(el, list) for el in sample[0])) # prints true if any elements in sample[0] are a list
    
    sample[0] = (sample[0])[:15000]
    sample[1] = (sample[1])[:15000]
    sample[2] = (sample[2])[:15000]
    
    print(len(sample[0]))
    print(len(sample[1]))
    print(len(sample[2]))
    
    test = [transform('C:\\Users\\James Hoffman\\Desktop\\Speech_Recognition\\test_files\\1\\bbal4n.wav')]
    test = [transform('C:\\Users\\James Hoffman\\Desktop\\Speech_Recognition\\test_files\\3\\bbal4n.wav')]
    test[0] = (test[0])[:15000]
    
    kmeans = KMeans(n_clusters=3, random_state=0).fit(sample)
    pred_labels = kmeans.predict(test)
    print(pred_labels)
    print(kmeans.labels_)

In [36]:
main()

iterating through directory:  1
(257, 60)
(15420,)
(257, 62)
(15934,)
(257, 81)
(20817,)
(257, 60)
(15420,)
(257, 67)
(17219,)
(257, 65)
(16705,)
(257, 66)
(16962,)
(257, 70)
(17990,)
(257, 66)
(16962,)
(257, 73)
(18761,)
iterating through directory:  2
(257, 82)
(21074,)
(257, 84)
(21588,)
(257, 84)
(21588,)
(257, 89)
(22873,)
(257, 78)
(20046,)
(257, 94)
(24158,)
(257, 92)
(23644,)
(257, 89)
(22873,)
(257, 94)
(24158,)
(257, 98)
(25186,)
iterating through directory:  3
(257, 112)
(28784,)
(257, 99)
(25443,)
(257, 89)
(22873,)
(257, 98)
(25186,)
(257, 101)
(25957,)


KeyboardInterrupt: 